In [30]:
# @hidden_cell
# import packages
from __future__ import print_function # partially (or only) imports the print_function method of numpy

import pandas as pd # import panda and assign alias pd
import numpy as np # import numpy and assign alias np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### pip(optionally) or conda-forge/packages installation on local machine and/or on the cloud (i.e., IBM Watson) 
## for formatting purposes.
# !conda update nbformat
# !conda update nbformat
## Generic syntax highlighter suitable for use in code hosting, forums, wikis or other applications that need to prettify source code
# !conda install -c anaconda pygments 
## provides a command line tool to convert Jupyter notebook files to HTML files - https://www.nbinteract.com/
# !pip install nbinteract
## An interactive, browser-based graphing library for Python
# !conda install -c conda-forge plotly
## Jupyter Interactive Widgets
# !conda install -c conda-forge ipywidgets
## Make beautiful maps with Leaflet.js and Python
# !conda install -c conda-forge folium
## Rendering of live Jupyter notebooks with interactive widgets - https://anaconda.org/conda-forge/voila
# !conda install -c conda-forge voila
## Gridstack template for voila - https://anaconda.org/conda-forge/voila-gridstack
# !conda install -c conda-forge voila-gridstack
## Vuetify specific template for Voila - https://anaconda.org/conda-forge/voila-vuetify
# !conda install -c conda-forge voila-vuetify
## Reveal template for voila - https://anaconda.org/conda-forge/voila-reveal
# !conda install -c conda-forge voila-reveal
## Material template for Voila - https://anaconda.org/conda-forge/voila-material
# !conda install -c conda-forge voila-material
## A collection of various different notebook extensions for Jupyter - https://anaconda.org/conda-forge/jupyter_contrib_nbextensions
# !conda install -c conda-forge jupyter_contrib_nbextensions

In [16]:
# Data Loading 
# Johns Hopkins CSV interaction using JHU CSSE COVID-19 
# Dataset Time series summary (csse_covid_19_time_series)
death_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
confirmed_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
recovered_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
country_df = pd.read_csv("https://raw.githubusercontent.com/cssegisanddata/covid-19/web-data/data/cases_country.csv")

In [17]:
# Data preparation
death_df.head();

In [18]:
# Data preparation
confirmed_df.head();

In [19]:
# Data preparation
recovered_df.head();

In [20]:
# Data preparation
country_df.head();

In [21]:
# data cleaning - renaming
country_df.columns = map(str.lower, country_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)
death_df.columns = map(str.lower, death_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)

In [22]:
# data cleaning - renaming
confirmed_df = confirmed_df.rename(columns = {'province/state': 'state', 'country/region': 'country'})
recovered_df = recovered_df.rename(columns = {'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns = {'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns = {'country_region': 'country'})

In [23]:
# data cleaning - renaming
sorted_country_df = country_df.sort_values('confirmed', ascending=False).head(3)
sorted_country_df;

In [24]:
# Data preparation
def highlight_col(x):
    r = 'background-color: red'
    p = 'background-color: purple'
    g = 'background-color: grey'
    temp_df = pd.DataFrame('', index=x.index, columns = x.columns)
    temp_df.iloc[:,4] = p
    temp_df.iloc[:,5] = r
    temp_df.iloc[:,6] = g
    return temp_df

sorted_country_df.style.apply(highlight_col, axis=None)


,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
17,US,2020-05-06 23:32:28,40.000000,-100.000000,1227430,73095,189910,990334,372.551021,nan,nan,5.955126,840,USA
161,Spain,2020-05-06 23:32:28,40.463667,-3.749220,220325,25857,126002,68466,471.235210,nan,nan,11.735845,724,ESP
10,Italy,2020-05-06 23:32:28,41.871900,12.567400,214457,29684,93245,91528,354.698174,nan,nan,13.841469,380,ITA


In [25]:
# Data preparation
# x and y given as array_like objects
import plotly.express as px
fig = px.scatter(sorted_country_df.head(10), x='country', y='confirmed', size='confirmed', 
                  color='country', hover_name='country', size_max=60)
# fig.update_layout()
fig.show()

In [26]:
import plotly.graph_objects as go

def plot_cases_for_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4,5]

    df_list = [confirmed_df, death_df]

    fig = go.Figure()

    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 5:].columns))
            y_data = np.sum(np.asarray(df.iloc[:, 5:]), axis=0)
        
        else:
            x_data = np.array(list(df.iloc[:, 5:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,5:]), axis=0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
                                    name=labels[i],
                                    line = dict(color=colors[i], width=line_size[i]),
                                    connectgaps=True,
                                    text = "Total " + str(labels[i]) + ": "+ str(y_data[-1])
                                ))

    fig.show()
        
# plot_cases_for_country('China')

interact(plot_cases_for_country, country='World');

interactive(children=(Text(value='World', description='country'), Output()), _dom_classes=('widget-interact',)…

In [27]:
import folium

world_map = folium.Map(location=[11,0], tiles="cartodbpositron", 
                       zoom_start=2, max_zoom=6, min_zoom=2)

for i in range(len(confirmed_df)):
    folium.Circle(
    location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
              fill = True,
              radius = (int((np.log(confirmed_df.iloc[i,-1]+1.0001))) + 0.2)*50000,
              fill_color = 'blue',
              color = 'red',
              tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                            "<h4 style='text-align:center;font-weight:bold'>"+confirmed_df.iloc[i]['country'] +"</h4>"+
                            "<hr style='margin:10px;color:white;'>"+
                            "<ul style='color:white;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                            "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                            "<li>Deaths: "+str(death_df.iloc[i,-1])+"</li>"+
                            "<li>Death Rate: "+str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                        "</ul></div>",
              ).add_to(world_map)
              
world_map

In [31]:
import nbinteract as nbi
nbi.publish('kenaitian/voila/master', 'DataVisualization.ipynb');

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... Saved 'DataVisualization.ipynb'.
Converting notebook...



Successfully converted!

<a href="DataVisualization.html" target="_blank" download>Click to download your webpage.</a>

To host your webpage, see the documentation:
<a href="https://www.nbinteract.com/tutorial/tutorial_publishing.html"
        target="_blank">
    https://www.nbinteract.com/tutorial/tutorial_publishing.html
</a>
